In [1]:
!pip install tensorflow

     |████████████████████████████████| 252.5 MB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.2 MB/s  eta 0:00:01
     |████████████████████████████████| 417 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 25.8 MB 9.9 MB/s eta 0:00:011
     |████████████████████████████████| 1.3 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 135 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 5.5 MB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 1.3 MB/s eta 0:00:011
     |████████████████████████████████| 3.5 MB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 667 kB 92.4 MB/s eta 0:00:01
     |████████████████████████████████| 72 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 243 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████

In [2]:
!pip install tensorflow-metal

     |████████████████████████████████| 1.4 MB 1.4 MB/s eta 0:00:01
You should consider upgrading via the '/Users/ron/Documents/github/myenv/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import tensorflow as tf

/Users/ron/Documents/github/myenv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}


In [5]:
cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 57s 0us/step


In [9]:
x_train.shape

(50000, 32, 32, 3)

In [10]:
x_test.shape

(10000, 32, 32, 3)

In [6]:
model = tf.keras.applications.ResNet50(
  include_top=True,
  weights=None,
  input_shape=(32, 32, 3),
  classes=100,)

2025-05-06 17:23:20.515873: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2025-05-06 17:23:20.515960: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2025-05-06 17:23:20.515978: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
I0000 00:00:1746577400.516065  536027 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1746577400.516162  536027 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 38, 38, 3) │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 16, 16,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 16, 16,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 16, 16,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 18, 18,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 8, 8, 64)  │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 8, 8, 64)  │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_1_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 8, 8, 64)  │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 8, 8, 64)  │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 8, 8, 64)  │          0 │ conv2_block1_2_b… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 8, 8, 256) │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 8, 8, 256) │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 8, 8, 256) │      1,024 │ conv2_block1_3_c

 Total params: 23,792,612 (90.76 MB)

 Trainable params: 23,739,492 (90.56 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [11]:
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

In [12]:
model.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5


2025-05-06 17:25:40.883161: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


782/782 ━━━━━━━━━━━━━━━━━━━━ 217s 252ms/step - accuracy: 0.0517 - loss: 5.1179
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 165s 211ms/step - accuracy: 0.1176 - loss: 4.1001
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 167s 213ms/step - accuracy: 0.1683 - loss: 3.7293
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 168s 215ms/step - accuracy: 0.1777 - loss: 3.7601
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 167s 214ms/step - accuracy: 0.1665 - loss: 3.6524


In [32]:
# Make a prediction
y_prediction = []
import numpy as np
prediction_p = model.predict(x_test)
y_pred = tf.nn.softmax(prediction_p)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step


In [33]:
sum(y_pred[0])

<tf.Tensor: shape=(), dtype=float32, numpy=1.0000001192092896>

In [34]:
y_pred

<tf.Tensor: shape=(10000, 100), dtype=float32, numpy=
array([[0.00989721, 0.00990425, 0.00992171, ..., 0.00992798, 0.00990478,
        0.01053685],
       [0.01000082, 0.01012609, 0.01017535, ..., 0.00994487, 0.00996232,
        0.01010393],
       [0.00998299, 0.01030684, 0.01001987, ..., 0.00991991, 0.00993217,
        0.00994131],
       ...,
       [0.01100329, 0.00992377, 0.00989089, ..., 0.0098863 , 0.0098872 ,
        0.00991733],
       [0.00990737, 0.0099593 , 0.00998165, ..., 0.01027233, 0.00995912,
        0.01007156],
       [0.01083752, 0.01026524, 0.01018604, ..., 0.00989836, 0.01010202,
        0.00992608]], dtype=float32)>

In [38]:
y_prediction = [0]* len(y_pred)
for i in range(len(y_pred)):
    y_prediction[i] = np.argmax(y_pred[i])

In [40]:
y_prediction[0:5]

[np.int64(12), np.int64(28), np.int64(69), np.int64(22), np.int64(62)]

In [41]:
y_test[5]

array([92])

In [ ]:
#tf.nn.softmax(prediction_p)

In [42]:
np.mean(np.array(y_test) == np.array(y_prediction)) * 100

np.float64(1.0)